<a href="https://colab.research.google.com/github/TaeMD/ML-projects/blob/main/Covid19_Pneumonia_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/education454/datasets.git

Cloning into 'datasets'...
remote: Enumerating objects: 2301, done.
remote: Total 2301 (delta 0), reused 0 (delta 0), pack-reused 2301
Receiving objects: 100% (2301/2301), 1.31 GiB | 46.69 MiB/s, done.
Checking out files: 100% (2295/2295), done.


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random

In [3]:
base_dir = '/content/datasets/Data'
base_train_covid = os.path.join(base_dir, 'train/COVID19')
base_train_normal = os.path.join(base_dir, 'train/NORMAL') 

In [4]:
normal_length = len(os.listdir(base_train_normal))
covid_length = len(os.listdir(base_train_covid))

print('normal cases:', len(os.listdir(base_train_normal)))
print('covid cases:', len(os.listdir(base_train_covid)))

normal cases: 1266
covid cases: 545


Split base_train into train and validation

NORMAL CXR

In [5]:
nsplit = int(normal_length * 0.8)
csplit = int(covid_length * 0.8)
print(nsplit)
print(csplit)

1012
436


In [10]:
normal_file_list = os.listdir(base_train_normal)

In [11]:
random.shuffle(normal_file_list)

train_normal_file_list = normal_file_list [:nsplit]
validation_normal_file_list = normal_file_list [nsplit:]
print(len(train_normal_file_list))
print(len(validation_normal_file_list))

1012
254


In [12]:
validation_dir = base_dir + '/validation'
validation_normal_dir = validation_dir + '/NORMAL'
validation_covid_dir = validation_dir + '/COVID19'

os.mkdir(validation_dir)
os.mkdir(validation_normal_dir)
os.mkdir(validation_covid_dir)

In [13]:
validation_normal_dir

'/content/datasets/Data/validation/NORMAL'

In [14]:
base_train_normal

'/content/datasets/Data/train/NORMAL'

In [15]:
for f in validation_normal_file_list:
  os.rename (os.path.join(base_train_normal,f), os.path.join(validation_normal_dir,f))

In [16]:
len(os.listdir(validation_normal_dir))

254

COVID CXR

In [17]:
covid_file_list = os.listdir(base_train_covid)

In [19]:
len(covid_file_list)

545

In [20]:
random.shuffle(covid_file_list)

train_covid_file_list = covid_file_list [:csplit]
validation_covid_file_list = covid_file_list [csplit:]
print(len(train_covid_file_list))
print(len(validation_covid_file_list))

436
109


In [22]:
for f in validation_covid_file_list:
  os.rename (os.path.join(base_train_covid, f), os.path.join(validation_covid_dir, f))

In [23]:
len(os.listdir(validation_covid_dir))

109